In [ ]:
import csv
import numpy as np
X = []
with open("./data/train/crawler/data/tweets.txt.text", newline='', encoding='utf8') as file_data:
    i = 0 
    for row in file_data:
        if i < 500:
            i = i +1
            X.append(row)

y = []
with open("./data/train/crawler/data/tweets.txt.labels", newline='', encoding='utf8') as file_data:
    j = 0
    for row in file_data:
        if j < 500:
            j = j +1
            y.append(row.replace("\n",""))


In [ ]:
X = np.array(X)
y = np.array(y)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()
X_dtm = vect.fit_transform(X)

In [ ]:
test=[]
with open("./data/test/us_test.text", newline='', encoding='utf8') as test_data:
    file = test_data.readlines()
    for row in file:
        test.append(row.replace("\n",""))

test = np.asarray(test)
test_dtm = vect.transform(test)

In [ ]:
test_label = []
with open("./data/test/us_test.labels", newline='', encoding='utf8') as test_data_label:
    file = test_data_label.readlines()
    for row in file:
        test_label.append(row.replace("\n",""))
test_label = np.asarray(test_label)
test_label = test_label.reshape(-1,1)

In [ ]:
import pandas as pd
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import cross_val_score

models = [
    SGDClassifier(loss=log, penalty=l2, alpha=0.001, l1_ratio=0.15, fit_intercept=True, max_iter=1000, 
                    tol=0.001, epsilon=0.1, n_jobs=-1, validation_fraction=0.1, 
                    n_iter_no_change=5)

]

CV = 5
cv_df = pd.DataFrame(index=range(CV * len(models)))
entries = []

for model in models:
    model_name = model.__class__.__name__
    model.fit(X_dtm, y)
    model.predict(test_dtm)
    score = model.score(test_dtm, test_label)
    entries.append((model_name, score))

cv_df = pd.DataFrame(entries, columns=['model_name', 'score'])

In [ ]:
print(cv_df)

In [ ]:
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import SGDClassifier

# Best with 'alpha': 0.001, 'loss': 'log', 'penalty': 'l2'
parameters = {'loss':('hinge', 'log', 'modified_huber'), 'penalty':('l2', 'elasticnet'), 'alpha':[0.0001, 0.001], }
sgd = SGDClassifier(l1_ratio=0.15, fit_intercept=True, max_iter=1000, 
                    tol=0.001, epsilon=0.1, n_jobs=-1, validation_fraction=0.1, 
                    n_iter_no_change=5)

clf = GridSearchCV(sgd, parameters, cv=3, n_jobs=-1)

clf.fit(X_dtm, y)

print(clf.cv_results_)